In [ ]:
import pandas as pd
import numpy as np
import re
import requests  # 🛰️ API für Wetterdaten abrufen
from sklearn.linear_model import LinearRegression
from datetime import datetime
from langchain.tools import Tool  # Agent-Tool für KI-Agenten (LangChain)
from langchain.agents import initialize_agent  # 🧠 AI-Agent

# 1️⃣ Historische Daten für Klimageräte (Trainingsdaten)
np.random.seed(42)
date_range = pd.date_range(start="2023-01-01", end="2023-12-31", freq="D")
sales_data = np.random.poisson(lam=15, size=len(date_range))  # Klimageräte-Verkaufszahlen

sales_df = pd.DataFrame({
    'Date': date_range,
    'Sales': sales_data
})
sales_df['Date'] = pd.to_datetime(sales_df['Date'])
sales_df.set_index('Date', inplace=True)

# 2️⃣ Dummy-Klimadaten (Monatliche Temperaturdurchschnitte)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
temperature_data = [0, 2, 8, 15, 20, 25, 30, 29, 24, 16, 8, 2]  # Temperatur in °C

climate_data = pd.DataFrame({
    'Month': months,
    'Average Temperature (°C)': temperature_data
})
climate_data['Month'] = climate_data['Month'].apply(lambda x: datetime.strptime(x, "%b").month)

# 3️⃣ Historische Klimadaten mit Verkaufsdaten verknüpfen
sales_df['Month'] = sales_df.index.month
data = pd.merge(sales_df.reset_index(), climate_data, on='Month', how='inner')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data['Weekday'] = data.index.weekday  # Wochentag als Feature

# 4️⃣ Maschinelles Lernmodell trainieren
X = data[['Average Temperature (°C)', 'Weekday']]
y = data['Sales']
model = LinearRegression()
model.fit(X, y)

# 5️⃣ **Agentic Feature: Automatische Wetterdatenerfassung**
def get_live_temperature(city="Berlin"):
    """Holt aktuelle Temperatur von einer Wetter-API (Ersatz durch OpenWeather oder andere API)."""
    API_KEY = "your_openweather_api_key"  # 🔑 Eigener API Key nötig
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['main']['temp']  # Rückgabe: Temperatur in °C
    else:
        return None

# 6️⃣ **Agent-Funktion für Klimageräte-Prognose**
def predict_demand(temp, weekday):
    """Agent berechnet die Nachfrage nach Klimageräten basierend auf Temperatur & Wochentag."""
    return model.predict(np.array([[temp, weekday]]))[0]

# 7️⃣ **Agent nutzt Tools & entscheidet, ob Live-Daten erforderlich sind**
def agentic_prediction(city, month_name, year):
    """Agent ruft Live-Daten ab & kombiniert mit historischen Daten für beste Prognose."""
    
    # 📌 Schritt 1: Live-Wetter abrufen
    live_temp = get_live_temperature(city)
    
    # 📌 Schritt 2: Historische Temperatur als Fallback
    month_index = months.index(month_name)
    historical_temp = climate_data['Average Temperature (°C)'][month_index]

    # 📌 Schritt 3: Agent entscheidet, ob Live-Daten verwendet werden sollen
    if live_temp:
        temp = (live_temp + historical_temp) / 2  # Kombinierte Prognose
    else:
        temp = historical_temp  # Falls keine Live-Daten verfügbar sind
    
    weekday = 0  # Annahme: Montag
    prediction = predict_demand(temp, weekday)

    # 📌 Schritt 4: Antwort generieren
    response = f"📊 **Prognose für Klimageräte in {city} ({month_name} {year})**:\n"
    response += f"   🔹 Erwartete Nachfrage: **{prediction:.2f} Klimageräte**\n"
    response += f"   🔹 Durchschnittstemperatur: {temp}°C (Basierend auf {'Live' if live_temp else 'historischen'} Daten)\n"
    print(response)

# 8️⃣ **Agent-Chatbot für interaktive Anfragen**
def chatbot():
    print("🤖 **Willkommen beim AI-Agenten für Klimageräte-Prognosen!**")
    
    user_input = input("❓ Deine Frage: ").strip()
    match = re.search(r"([a-zA-Z]+)\s*(\d{4})", user_input)
    
    if match:
        month_name = match.group(1)[:3].capitalize()
        year = int(match.group(2))
        
        city = input("🌍 Gib eine Stadt für Echtzeit-Wetter ein (oder drücke Enter für Standard: Berlin): ").strip()
        city = city if city else "Berlin"

        agentic_prediction(city, month_name, year)
    else:
        print("⚠️ Bitte gib eine Frage im Format 'Monat Jahr' ein, z. B. 'Juli 2025'.")

# **Startet den AI-Agenten für Klimageräte-Prognosen**
chatbot()
